# **Parseo de documentos y validación con Pydantic** 🔧📋  

Este notebook forma parte del curso de **IA Generativa** de la [Fundación GoodJob](https://www.fundaciongoodjob.org/).  

En este bloque vamos a recorrer un camino que comienza con los **datos en bruto** ; documentos no estructurados como PDFs, páginas web o respuestas de APIs— y termina con **objetos Python limpios y validados**, listos para usarse en cualquier aplicación.  

<div style="display: flex; justify-content: space-around; align-items: center; margin: 20px 0;">
  <img src="../notebooks/sources/llamaindex.png" alt="LlamaIndex" style="width:40%; height:auto;"/>
  <span style="font-size: 2em; font-weight: bold;">+</span>
  <img src="../notebooks/sources/pydantic.jpg" alt="Pydantic" style="width:40%; height:auto;"/>
</div>  

Para ello, nos apoyaremos en dos piezas clave:  
- **LlamaIndex**, que nos permitirá leer y transformar documentos de muy distinta naturaleza en un formato unificado y manejable.  
- **Pydantic**, que nos ayudará a garantizar que la información extraída sigue un esquema coherente, evitando errores y asegurando consistencia en todo el pipeline.  

La idea es entender cómo se combinan estas herramientas. Primero veremos cómo LlamaIndex actúa como capa de ingesta y parsing, capaz de convertir tanto documentos locales como datos de una API en representaciones estructuradas. A continuación, abordaremos la necesidad de controlar lo que devuelve un modelo: los LLMs son potentes, pero también pueden ser inconsistentes en la forma en que entregan resultados. Ahí es donde entra **Pydantic**, imponiendo reglas claras sobre qué forma deben tener los datos finales.  

Al final de este recorrido habremos construido un **flujo completo**: desde la carga de documentos y APIs, pasando por su procesamiento con LlamaIndex, hasta la validación final con Pydantic. No se trata solo de “leer datos”, sino de transformarlos en conocimiento fiable y utilizable dentro de un proyecto real.  


## 1. **Contexto: ¿Por qué parsear documentos con LlamaIndex?** 🤔

### **El problema típico**

En el mundo real, los datos no vienen en formatos perfectos. Nos enfrentamos a:

- **📄 PDFs** con texto desestructurado
- **🌐 HTML** con información dispersa  
- **📧 Emails** con formatos inconsistentes
- **📊 Documentos financieros** con tablas y gráficos

### **¿Qué ofrece LlamaIndex?**

LlamaIndex convierte todos estos documentos desestructurados en **nodes** que los modelos de lenguaje pueden consumir eficientemente.

```
Documento desestructurado → LlamaIndex → Nodes → Modelo LLM → Respuesta estructurada
```

### **La ventaja clave: Unificación**

No importa si la fuente es:
- Un PDF de 100 páginas
- Una API REST con datos financieros  
- Un sitio web con noticias

**El pipeline es consistente** → mismo código, misma lógica, mismos resultados.

---


## 2. **Parseo de documentos con LlamaIndex** 📚

### **Conceptos fundamentales**

LlamaIndex trabaja con dos abstracciones principales:

- **`Document`**: Representa un documento completo (PDF, HTML, texto)
- **`Node`**: Fragmentos pequeños de información que el modelo puede procesar

```python
# Flujo típico
Document → Splitter → [Node, Node, Node, ...] → Vector Store → Query Engine
```

### **Conectores disponibles**

LlamaIndex incluye múltiples conectores para diferentes fuentes:

| Conector | Uso | Ejemplo |
|----------|-----|---------|
| `SimpleDirectoryReader` | Archivos locales | PDFs, TXT, DOCX |
| `PDFReader` | PDFs específicos | Reportes financieros |
| `WebPageReader` | Páginas web | Artículos, blogs |
| `WikipediaReader` | Wikipedia | Datos enciclopédicos |
| `NotionPageReader` | Notion | Documentación |
| `...`| ... | ...|

Si bien los conectores mencionados en la tabla anterior son algunos de los más importantes y versátiles —especialmente `SimpleDirectoryReader`, `PDFReader` y `WebPageReader`—, existen muchos otros disponibles que pueden adaptarse a diferentes necesidades y fuentes de datos.

Puedes explorar la lista completa de conectores y ejemplos de uso en la [documentación oficial de LlamaIndex](https://docs.llamaindex.ai/en/stable/examples/).

### **Metadatos: la clave del contexto**

Cada `Node` no solo contiene texto, sino también metadatos asociados que enriquecen su valor. Estos metadatos son atributos que describen el fragmento y permiten a los modelos trabajar con información más precisa y relevante.
```python
metadata = {
    "fecha": "2024-03-15",
    "fuente": "api_financiera", 
    "autor": "analyst_bot",
    "categoria": "earnings_report"
}
```

### ⭐ ¿Por qué son importantes los metadatos?

| Icono | Función | Ejemplo |
|-------|---------|---------|
| 🧭 **Contextualización** | Explican de dónde proviene la información y en qué condiciones fue generada. | *“Informe de resultados trimestral publicado en marzo de 2024”* |
| 🔍 **Filtrado inteligente** | Permiten restringir consultas según criterios específicos. | *“Buscar solo artículos posteriores a 2022”* <br> *“usar únicamente la fuente `api_financiera`”* |
| ⚖️ **Priorización y relevancia** | Diferencian nodos con contenido similar en función de su origen o fiabilidad. | *Dar más peso a un documento oficial que a un blog anónimo* |
| 📑 **Trazabilidad** | Cada respuesta puede rastrearse hasta el documento de origen. | *Clave en entornos regulados donde se exige justificar la procedencia* |

---

✨ En conjunto, los metadatos convierten texto plano en **información estructurada, explicable y confiable**.  



In [5]:
# Imports necesarios
import os
import json
import requests
from datetime import datetime
from typing import List, Optional, Dict, Any
from pathlib import Path

# LlamaIndex imports
from llama_index.core import (
    Document, 
    VectorStoreIndex, 
    SimpleDirectoryReader,
    Settings
)
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import NodeWithScore
from llama_index.readers.web import SimpleWebPageReader
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Pydantic imports
from pydantic import BaseModel, Field, validator
from pydantic.types import constr
from enum import Enum

# Configurar LlamaIndex
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.0)
Settings.embed_model = OpenAIEmbedding()

print("✅ Configuración completada")
print(f"📚 LlamaIndex configurado con {Settings.llm.model}")
print(f"📚 LlamaIndex Embedding configurado con {Settings.embed_model.model_name}")

print("🔧 Pydantic listo para validación")


✅ Configuración completada
📚 LlamaIndex configurado con gpt-4o-mini
📚 LlamaIndex Embedding configurado con text-embedding-ada-002
🔧 Pydantic listo para validación


### **2.1 SimpleDirectoryReader: Carga de documentos**

El componente `SimpleDirectoryReader` constituye uno de los mecanismos más versátiles de LlamaIndex para la ingesta de datos.  
De acuerdo con la [documentación oficial](https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/), este lector no se limita a archivos de texto plano, sino que admite múltiples formatos habituales en entornos profesionales y de investigación.  

---

#### 📁 **Tipos de archivos soportados**

| Extensión           | Tipo de contenido    | Uso común                                   |
|---------------------|----------------------|---------------------------------------------|
| `.csv`              | Datos tabulares      | Datasets, reportes financieros              |
| `.pdf`              | Documento PDF        | Contratos, informes técnicos o académicos   |
| `.docx`             | Documento Word       | Documentación corporativa                   |
| `.md`               | Markdown             | Manuales y documentación técnica            |
| `.ipynb`            | Jupyter Notebook     | Notebooks de análisis y experimentación     |
| `.jpeg/.jpg/.png`   | Imágenes             | OCR, procesamiento de contenido visual      |
| `.mp3/.mp4`         | Audio y vídeo        | Generación de transcripciones automáticas   |

---

#### 🏷️ **Metadatos asociados a cada documento**

Además del contenido textual, `SimpleDirectoryReader` enriquece cada documento con un conjunto de metadatos normalizados.  
Estos metadatos facilitan tareas de filtrado, trazabilidad y organización dentro de un pipeline de procesamiento. Entre los más relevantes se incluyen:  

- **`file_path`**: ruta completa del archivo en el sistema de ficheros.  
- **`file_name`**: nombre original del archivo.  
- **`file_type`**: tipo MIME detectado automáticamente.  
- **`file_size`**: tamaño del archivo en bytes.  
- **`creation_date`** y **`last_modified_date`**: fechas de creación y última modificación, normalizadas a UTC.  

---

En consecuencia, `SimpleDirectoryReader` debe considerarse no solo como un cargador de documentos, sino como una **herramienta de ingesta integral**, capaz de proporcionar tanto el contenido como el contexto necesario para un procesamiento fiable con modelos de lenguaje.  


In [29]:
# Ejemplo 1: Carga simple de documentos locales
data_path = Path("../data")

# Verificar qué archivos tenemos
if data_path.exists():
    files = list(data_path.glob("*.csv"))
    print("📁 Archivos encontrados:")
    for file in files:
        print(f"  - {file.name}")
else:
    print("❌ Directorio data/ no encontrado")

# Cargar documentos con SimpleDirectoryReader
reader = SimpleDirectoryReader(
            input_dir="../data",
            recursive=False,  # Buscar en subdirectorios
            required_exts=[".csv", ".txt", ".md", ".pdf"],  # Solo estos tipos
            num_files_limit=10,  # Máximo 10 archivos para el ejemplo
            encoding="utf-8"  # Codificación específica
        )

documents = reader.load_data(num_workers=4)

print(f"\n✅ Cargados {len(documents)} documentos")

# Inspeccionar el primer documento
if documents:
    for doc, tipo in zip(documents, ["CSV","PDF"]):
        print(f"\n📄 Documento {tipo}:")
        print(f"  - ID: {doc.doc_id}")
        print(f"  - Metadatos: {doc.metadata}")
    print(f"  - Texto (primeros 200 chars): {doc.text[:200]}...")
        


📁 Archivos encontrados:
  - partner_headlines_sample.csv

✅ Cargados 40 documentos

📄 Documento CSV:
  - ID: 816c911d-fd21-4b31-aa28-8584e20d8c42
  - Metadatos: {'page_label': '1', 'file_name': 'BOE-A-1978-31229-consolidado.pdf', 'file_path': 'c:\\Users\\Alejandro\\OneDrive - Universidad de Oviedo\\Ciencia e Ingeniería de Datos\\Personal\\GoodJob\\curso-llamaindex-pydantic\\notebooks\\..\\data\\BOE-A-1978-31229-consolidado.pdf', 'file_type': 'application/pdf', 'file_size': 329735, 'creation_date': '2025-09-10', 'last_modified_date': '2025-09-10'}

📄 Documento PDF:
  - ID: 015dc779-1569-4811-93f7-f7024f6bac34
  - Metadatos: {'page_label': '2', 'file_name': 'BOE-A-1978-31229-consolidado.pdf', 'file_path': 'c:\\Users\\Alejandro\\OneDrive - Universidad de Oviedo\\Ciencia e Ingeniería de Datos\\Personal\\GoodJob\\curso-llamaindex-pydantic\\notebooks\\..\\data\\BOE-A-1978-31229-consolidado.pdf', 'file_type': 'application/pdf', 'file_size': 329735, 'creation_date': '2025-09-10', 'last_modifie

### **2.2 WebPageReader: Parseo de páginas web**

El componente `WebPageReader` permite la ingesta de contenido directamente desde sitios web.  
De acuerdo con la [documentación oficial](https://docs.llamaindex.ai/en/stable/api_reference/readers/https://docs.llamaindex.ai/en/stable/examples/data_connectors/WebPageDemo/#using-simplewebpagereader), LlamaIndex recomienda en muchos casos el uso de servicios externos especializados en *web scraping* y *web crawling*.  
Esto se debe a que dichos servicios ofrecen mayor robustez frente a cambios en la estructura de las páginas, además de funcionalidades avanzadas como manejo de JavaScript dinámico, crawling masivo o almacenamiento en caché.  

No obstante, aunque no es el foco de este curso, es importante señalar que implementar procesos básicos de scraping directamente en Python no resulta complejo.  
Existen librerías ampliamente utilizadas que permiten recuperar y procesar el contenido de páginas web de manera sencilla, entre ellas:  

- **`requests`** → para realizar peticiones HTTP.  
- **`BeautifulSoup`** (`bs4`) → para el parseo y extracción de información del HTML.  
- **`playwright`** o **`selenium`** → cuando es necesario interactuar con páginas que renderizan contenido dinámico con JavaScript.  
- **`scrapy`** → casos de scraping más “serios” (crawling, control de concurrencia, reintentos, deduplicación)

De este modo, `WebPageReader` puede integrarse con estos enfoques: bien apoyándose en soluciones externas más completas, o bien complementándose con librerías estándar de Python para casos más simples.  

En definitiva, este lector convierte páginas web en nodos procesables, lo que amplía el abanico de fuentes de información que pueden incorporarse a un pipeline de IA.  


In [30]:
from llama_index.core import SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
from IPython.display import Markdown, display
import os

urls = []
with open("../llamaindex_docs_crawler/outputs/llamaindex_docs.jsonl", "r", encoding="utf-8") as f:
    for linea in f:
        urls.append(json.loads(linea)["url"])
documents = SimpleWebPageReader(html_to_text=True).load_data(
    urls[0:15] # Sample 15 pages
)
index = SummaryIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("What integrations does LlamaIndex support?")
display(Markdown(f"{response}"))


LlamaIndex supports a variety of integrations, including:

- **LlamaPacks**: Code templates for features that can be downloaded and edited.
- **Data Loaders**: Tools for loading data from various sources.
- **Agent Tools**: Tools that enhance the capabilities of agents.
- **LLMs**: Integration with numerous large language models.
- **Observability/Tracing/Evaluation**: Tools for monitoring and evaluating applications.
- **Experiment Tracking**: Integrations for tracking experiments.
- **Structured Outputs**: Tools for managing structured outputs.
- **Storage and Managed Indexes**: Options for storing and managing indexes.
- **Application Frameworks**: Support for frameworks like Streamlit and Chainlit.
- **Distributed Compute**: Integrations for distributed computing solutions.
- **Other**: Various additional integrations, including plugins for ChatGPT and other tools.

For a complete list and more details, you can explore LlamaHub, which hosts these integrations.

## 3. **Uso de APIs como fuentes de datos** 🌐

### Patrones de uso (¿cuándo indexar vs. llamar en tiempo real?)

- 🗄️ **Ingesta estática (→ índice vectorial):** llamas a la API, conviertes la respuesta en `Document`s y los indexas. Ideal para catálogos, FAQs o docs que cambian poco.

- ⚡ **Consulta dinámica (agente + tools):** el agente invoca la API en tiempo real (`FunctionTool`/`OpenAPI`). Útil para datos volátiles (clima, precios, métricas).

- 🔀 **Híbrido (RAG + herramientas):** un router decide: si la pregunta es del corpus → índice; si necesita frescura → API.


In [31]:
# Ejemplo 2: Cargar datos desde API pública
# Subir un directorio en sys.path para permitir importaciones relativas
import os, sys, json
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from src.apis.news_api import fetch_news

from llama_index.core import Document, VectorStoreIndex, Settings
from llama_index.core.node_parser import SentenceSplitter  # ← usamos SentenceSplitte

TOPICS_OF_INTEREST = ["AI", "Apple Inc.", "Tesla"]
all_docs = []

for interesting_topic in TOPICS_OF_INTEREST : 
    print(f"🌐 Buscando noticias para : {interesting_topic}")
    api_documents = fetch_news(query = interesting_topic)
    
    if api_documents:
        print(f"✅ Cargados {api_documents.total} documentos desde API")
        
        # Mostrar ejemplo
        articles = api_documents.articles
        if not articles : 
            print("Ups , No articles found ...")
            continue
        
        for article in articles : 
            print(f"\n📄 Documento desde API:")
            print(f"  - Fuente: {article.source}")
            print(f"  - URL: {article.url}")
            print(f"  - Fecha: {article.published_at}")
            print(f"  - Descripción: {article.description[:150]}...")
            
            payload = article.model_dump_json(by_alias=True, exclude_none=True)
            doc = Document(
            text=json.dumps(payload, ensure_ascii=False),
            metadata={
                "topic": interesting_topic,
                "source": article.source,
                "url": str(article.url),                         # <- HttpUrl → str
                "publishedAt": article.published_at.isoformat(), # <- datetime → ISO
                "title": article.title,
            },
        )
        all_docs.append(doc)

if not all_docs:
    print(" Nada que indexar. Revisa filtros/consulta de la API.")
else:
    # Segmentación por frases/chunks
    # Settings.node_parser = SentenceSplitter(chunk_size=1024, chunk_overlap=128)

    index = VectorStoreIndex.from_documents(all_docs)
    qe = index.as_query_engine(similarity_top_k=4)


🌐 Buscando noticias para : AI
✅ Cargados 10 documentos desde API

📄 Documento desde API:
  - Fuente: Redes-sociales.com
  - URL: https://www.redes-sociales.com/innovacion-acelerada-en-ifa-2025-pcs-copilot-y-windows-11/
  - Fecha: 2025-09-12 16:47:25+00:00
  - Descripción: Durante la más reciente edición de la IFA de Berlín 2025, Acer y Lenovo sorprendieron con innovaciones significativas dentro del campo tecnológico, pr...

📄 Documento desde API:
  - Fuente: El Financiero
  - URL: https://www.elfinanciero.com.mx/monterrey/2025/09/12/lanza-uanl-estrategia-de-ia-esencia/
  - Fecha: 2025-09-12 16:03:39+00:00
  - Descripción: La universidad incorpora cinco ejes estratégicos de inteligencia artificial para impulsar innovación, responsabilidad social y competitividad académic...

📄 Documento desde API:
  - Fuente: Nacion.com
  - URL: https://www.nacion.com/economia/microsoft-anuncia-inversion-en-capacidad/3Y2LMXEB45G2VOR22FHP7GL2V4/story/
  - Fecha: 2025-09-12 15:51:54+00:00
  - Descripción:

In [33]:
# Ejemplo de consulta (puedes cambiarla por algo ligado a tus noticias)
resp = qe.query("Dame un resumen sobre las ultimas noticas de apple")
print("\n🧠 Respuesta RAG:")
print(resp)


🧠 Respuesta RAG:
Recientemente, se ha reportado una experiencia negativa relacionada con el cambio de un Apple Watch defectuoso. Durante unas vacaciones en Calpe, un usuario enfrentó dificultades al intentar reemplazar su dispositivo, lo que convirtió su verano en una pesadilla. La situación destaca los problemas que algunos clientes pueden encontrar al gestionar productos defectuosos de la marca. Para más detalles, se puede consultar el artículo completo en Adslzone.net.


## 4. **Validación y estructuración con Pydantic** ✅

### **El problema: respuestas inconsistentes**

Los modelos de lenguaje pueden devolver respuestas en formatos variables:

```python
# Respuesta 1
"La empresa Apple ganó $123.9 billones en Q4"

# Respuesta 2  
"Ganancias: $123,900,000,000 (Q4 2023)"

# Respuesta 3
"Revenue for Apple in fourth quarter: 123.9B USD"
```



In [51]:
prompt = (
        "Dime los ingresos de Apple en el cuarto trimestre de 2023, "
        "pero responde con un formato estructurado json."
        "Devuelve unicamente el json sin ```json"
        )      
llm_text = Settings.llm.complete(prompt).text
samples = [
    'La empresa Apple ganó $123.9 billones en Q4',
    'Ganancias: $123,900,000,000 (Q4 2023)',
    'Revenue for Apple in fourth quarter: 123.9B USD',
]

for sam in samples : 
    llm_text = Settings.llm.complete(prompt + sam).text
    print(f"LLM_TEXT : {llm_text}")
    data = json.loads(llm_text)
    print(f"Json data : {data}")






LLM_TEXT : ```json
{
  "empresa": "Apple",
  "trimestre": "Q4 2023",
  "ingresos": 123.9,
  "unidad": "billones"
}
```


JSONDecodeError: Expecting value: line 1 column 1 (char 0)


### **La solución: Pydantic BaseModel**
Aunque los modelos de IA son cada vez más potentes y en ocasiones pueden devolver directamente un JSON válido, no debemos confiar ciegamente en ello.

En entornos reales, los LLM suelen introducir variaciones, errores de sintaxis o formatos ambiguos. Por eso, una buena práctica es tipar y validar la información en nuestro lado de la aplicación.

Pydantic nos permite definir **schemas estrictos** que garantizan:

- ✅ **Tipos de datos** correctos
- ✅ **Validaciones** personalizadas  
- ✅ **Formatos** consistentes
- ✅ **Errores** descriptivos

De esta manera, no importa cómo varíe la salida del LLM: nuestro sistema siempre recibe un objeto Python tipado, seguro y predecible.

In [1]:
# Ejemplo 3: Definir modelos Pydantic para validación
import os, sys, json
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from models.earnings_model import AnalisisFinanciero

# Ejemplo de uso del modelo
print("📋 Modelo Pydantic definido: AnalisisFinanciero")
print("\n🔍 Schema del modelo:")
print(json.dumps(AnalisisFinanciero.schema(), indent=2, ensure_ascii=False))


📋 Modelo Pydantic definido: AnalisisFinanciero

🔍 Schema del modelo:
{
  "$defs": {
    "SentimentoEnum": {
      "description": "Enum para sentimientos posibles",
      "enum": [
        "positivo",
        "negativo",
        "neutral"
      ],
      "title": "SentimentoEnum",
      "type": "string"
    }
  },
  "additionalProperties": false,
  "description": "Modelo para análisis de información financiera extraída de documentos",
  "properties": {
    "empresa": {
      "description": "Nombre de la empresa analizada",
      "maxLength": 100,
      "minLength": 1,
      "title": "Empresa",
      "type": "string"
    },
    "trimestre": {
      "description": "Trimestre del reporte (ej: Q4 2023)",
      "pattern": "Q[1-4]\\s\\d{4}",
      "title": "Trimestre",
      "type": "string"
    },
    "ingresos": {
      "description": "Ingresos en billones de USD",
      "exclusiveMinimum": 0,
      "title": "Ingresos",
      "type": "number"
    },
    "sentimiento": {
      "$ref": "#/$def

C:\Users\Alejandro\AppData\Local\Temp\ipykernel_14512\1930544089.py:9: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(json.dumps(AnalisisFinanciero.schema(), indent=2, ensure_ascii=False))


In [2]:

from llama_index.core import Settings, VectorStoreIndex, Document
from llama_index.readers.web import SimpleWebPageReader
apple_reports_urls = [
    "https://www.apple.com/newsroom/2025/07/apple-reports-third-quarter-results/",
    "https://www.apple.com/newsroom/2025/05/apple-reports-second-quarter-results/",
    "https://www.apple.com/newsroom/2025/01/apple-reports-first-quarter-results/",
    "https://www.apple.com/newsroom/2024/10/apple-reports-fourth-quarter-results/",
]
reader = SimpleWebPageReader(html_to_text=True, metadata_fn=lambda url: {"url": url})
docs = reader.load_data(apple_reports_urls)

index = VectorStoreIndex.from_documents(
    docs,
)

# 4) Query engine con parser tipado
qe = index.as_query_engine(
    output_cls=AnalisisFinanciero,          # <- fuerza JSON parseable a AnalisisFinanciero
    response_mode="tree_summarize",
)


resp = qe.query("Extrae el análisis financiero de Apple para el tercer trimestre 2025 (Q3 2025).")
af: AnalisisFinanciero = resp  # ya viene parseado al modelo
print(af)

{"empresa":"Apple Inc.","trimestre":"Q3 2025","ingresos":94.0,"sentimiento":"positivo","puntos_clave":["June quarter revenue record with double-digit growth in iPhone, Mac, and Services","Quarterly diluted earnings per share of $1.57, up 12 percent year over year","Services revenue reaches new all-time high","Cash dividend declared of $0.26 per share of common stock","Live streaming of financial results conference call available"],"fecha_analisis":"2025-09-13"}


## 🎯 **Conclusiones y próximos pasos**

### **¿Qué hemos logrado hoy?**

1. **📄 Parseo inteligente de documentos**: Aprendiste a usar **LlamaIndex** para leer PDFs, páginas web y otros formatos con una sola API unificada.  
2. **🤖 LLMs al servicio del estructurado**: No solo indexamos texto, sino que aplicamos **consultas guiadas** para extraer campos concretos en formato JSON.  
3. **✅ Validación estricta con Pydantic**: Cada salida del LLM pasa por un **esquema tipado**, garantizando consistencia y evitando errores como listas demasiado largas o valores fuera del Enum.  
4. **🔄 Robustez con validadores**: Implementamos **fallbacks y normalización** para corregir automáticamente desviaciones comunes (ej. recortar bullets a máximo 5, coerción de strings a enums).  
5. **⚡ Pipeline escalable**: El mismo enfoque funciona para parsear 1 documento o miles, integrando APIs en tiempo real y almacenando metadata validada en los objetos `Document`.

### 🚀 **¿Por qué es relevante?**

Porque este patrón une lo mejor de los **LLMs** (capacidad de comprender y resumir documentos complejos) con la **seguridad tipada de Pydantic** (estructura confiable, lista para usarse en bases de datos, APIs o dashboards).  
Esto es especialmente útil en proyectos donde los datos no siguen un formato estructurado, donde **los agentes** devuelven formatos válidos.


### 🛣️ **Cierre del curso y próximos pasos**

Hoy hemos finalizado este curso con éxito. 🎉  
A lo largo de las sesiones hemos recorrido un camino completo:

- ✅ ✨ **Introducción a LLMs, agentes y tools**
- ✅ 🤖 **Sistemas multi-agente y ChromaDB**
- ✅ 🏗️ **Validación con Pydantic**
- ✅ 📄 **Parseo avanzado de documentos**
- ⬜ 🚀 **Despliegue en producción**

Con esto ya cuentas con las bases para diseñar **pipelines de extracción y validación robustos**, capaces de combinar **IA generativa** con **control estructural** para aplicaciones confiables en entornos reales.

---

**¡Felicidades a tod@s! 🎉**  
Ahora toca avanzar y profundizar en el resto de sesiones del programa, donde exploraremos herramientas como  **n8n** y temas avanzados como  **MCP** y **A2A protocol**. 🚀


